In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA']="ON"

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import pandas as pd
df =pd.read_csv("/content/drive/MyDrive/severity_project/Disambiguation_results/pretrained_zeroshot-disambiguation-results.csv")
df.head()

,Filename,Text,ACTUAL,PREDICTED
0,28353604_df.csv,A 69-year-old man became aware of the onset of...,survivor,survivor
1,28559815_df.csv,"A 34-year-old woman, 1 week postpartum, presen...",survivor,survivor
2,28538413_df.csv,A 63-year-old male patient without smoking or ...,survivor,survivor
3,28353588_df.csv,A 75-year-old man was referred to our hospital...,survivor,death
4,28353596_df.csv,A 47-year-old female patient presented progres...,survivor,survivor


In [ ]:
#use this for only macrobatt dataset
df = df[0:200]
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['Actual'] = LE.fit_transform(df['ACTUAL'])
df['Actual'].value_counts()

1    169
0     31
Name: Actual, dtype: int64

In [ ]:
# possible_labels = df.Actual.unique()

# label_dict = {}
# for index, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = index

label_dict =dict( [(key,value) for key,value in enumerate(range(0,2))])
label_dict

{0: 0, 1: 1}

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.Text.values,
                                                  df.Actual.values,
                                                  test_size=0.10,
                                                  stratify=df.Actual.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

# df.loc[X_train, 'data_type'] = 'train'
# df.loc[X_val, 'data_type'] = 'val'

In [ ]:
def mark_as_train(row):
    if row['Text'] in X_train:
        return "train"
    return row['data_type']

# Apply the function to each row of the DataFrame using df.apply
df['data_type'] = df.apply(mark_as_train, axis=1)

In [ ]:
def mark_as_val(row):
    if row['Text'] in X_val:
        return "val"
    return row['data_type']

# Apply the function to each row of the DataFrame using df.apply
df['data_type'] = df.apply(mark_as_val, axis=1)

In [ ]:
# #for training
# for value in X_train :
#   for index in range(len(df)):
#     if value==df.iloc[index]['medical_abstract']:
#       df.loc[index,'data_type']="train"

# #for validation
# for value in X_val :
#   for index in range(len(df)):
#     if value==df.iloc[index]['medical_abstract']:
#       df.loc[index,'data_type']="val"
df.data_type.value_counts()

train    180
val       20
Name: data_type, dtype: int64

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=512,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Actual.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Actual.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(180, 20)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 5

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

In [ ]:

epochs = 50
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:

import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #save every 5 epoch
    if epoch %5==0:
      torch.save(model.state_dict(), f'/content/drive/MyDrive/severity_project/disambiguation_epochs/finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/36 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.49535786360502243
Validation loss: 0.48269279301166534
F1 Score (Weighted): 0.7810810810810811


Epoch 2:   0%|          | 0/36 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.43176971686383087
Validation loss: 0.45331988483667374
F1 Score (Weighted): 0.7810810810810811


...


Epoch 18
Training loss: 0.05012769589989653
Validation loss: 0.7487149753142148
F1 Score (Weighted): 0.81875


Epoch 19:   0%|          | 0/36 [00:00<?, ?it/s]


Epoch 19
Training loss: 0.04679899521741188
Validation loss: 0.824789522390347
F1 Score (Weighted): 0.81875


Epoch 20:   0%|          | 0/36 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Predictions

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/severity_project/disambiguation_epochs/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 1/3

Class: 1
Accuracy: 16/17



In [ ]:
df[df['data_type']=='val'].Actual.value_counts()

1    17
0     3
Name: Actual, dtype: int64

In [ ]:
df[df['data_type']=='val']

,Filename,Text,API KEY Prediction,Predicted,ACTUAL,Actual,data_type
10,28353561_df.csv,A 16-year-old female suffered from an abdomina...,LOW,survivor,survivor,1,val
31,28272214_df.csv,A 52-year-old man with a 30-year history of sm...,HIGH,survivor,survivor,1,val
32,28265107_df.csv,A 51-year-old G1P1 Caucasian female with lifel...,HIGH,survivor,survivor,1,val
49,28103924_df.csv,Our patient was a 7-year-old Italian boy born ...,LOW,survivor,survivor,1,val
54,28154287_df.csv,A 73-year-old man who had been diagnosed with ...,HIGH,survivor,death,0,val
58,28154669_df.csv,An 82-year-old male was admitted in our outsid...,HIGH,survivor,survivor,1,val
65,27683825_df.csv,"The proband is a 19-year-old woman, first chil...",MODERATE,survivor,survivor,1,val
97,26106249_df.csv,The patient was a 38-year-old Caucasian man of...,HIGH,survivor,survivor,1,val
100,26530965_df.csv,A 42-year-old woman presented with a right bre...,NEUTRAL,death,death,0,val
102,26445413_df.csv,A 33-year-old man presented with left flank du...,NEUTRAL,survivor,survivor,1,val
